In [2]:
# -*- coding: utf-8 -*-
import sys
import io
import time
import requests
import re
import json
import datetime
import urllib
from bs4 import BeautifulSoup
import sqlite3


# insta_comment

def insert_data(data):
    dbname = 'insta_comment(20.2.15~20.2.17).db'
    sql = """INSERT INTO food_table ('idx_text' , 'uesr_id' , 'date' , 'comments' ,'like' ,'keyword_food' )
            VALUES (? , ? , ? , ? , ? , ?)"""
    with sqlite3.connect(dbname) as conn:
        cur = conn.cursor()
        cur.execute( sql , data )
        conn.commit()
        
        
def exception_delay(json):
    global ERROR_COUNTS
    
    try:
        tmp = json['graphql']
    
        
        return False 
    
    except:
        ERROR_COUNTS += 1
        print('그만 좀 들어와라 5분 쉰다')
        time.sleep(100)
        
        return True
    
        
        
        
def text_data(final):
    review_text = final
    hangul = re.compile('[^ㄱ-ㅣ가-힣a-zA-Z#0-9]+')
    result=hangul.sub(" ",review_text)
    return result  


        


def get_first_json():
    global ERROR_COUNTS
    global MAX_ERROR
    global KEYWORD 
    
    is_go = True
    
    search = urllib.parse.quote(KEYWORD)
    
    
    while is_go:
        try:
            response = requests.get('https://www.instagram.com/explore/tags/' + search + '/?__a=1')
            json = response.json()
        except:
            print('막혔어 기다려')
            time.sleep(10)
            ERROR_COUNTS += 1
            
        is_go = exception_delay(json)
        
        if ERROR_COUNTS == MAX_ERROR:
            print('멈춘다 그만해라')
            return max_id
 

    try:
        max_id = json['graphql']['hashtag']['edge_hashtag_to_media']['edges'][len(json['graphql']['hashtag']['edge_hashtag_to_media']['edges']) - 1]['node']['id']
    except:
        print('끝났어 처음에')

    
    return max_id





def crawling_json(max_id):
    global ERROR_COUNTS
    global COUNT_PER_70
    global MAX_ERROR
    global DELAY_TIME
    global KEYWORD
    
    search = urllib.parse.quote(KEYWORD)

    headers = {'x-instagram-gis': 'ad034c3799ec2a9b083be2bbb257ffec',
               'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

    c = 0
    commet_counts = 0
    # while is_next:
    
    while c < COUNT_PER_70:
        
        time.sleep(DELAY_TIME)
        
        is_go = True
        
        while is_go:
            try:
                response2 = requests.get('https://www.instagram.com/explore/tags/' + search + '/?__a=1&max_id=' + max_id, headers = headers)
                json2 = response2.json()
            except:
                print('막혔어 기다려')
                json2 = None
                time.sleep(30)
                ERROR_COUNTS += 1
                
                
            is_go = exception_delay(json2)
            
            if ERROR_COUNTS >= MAX_ERROR:
                print('멈춘다 그만해라')
                return max_id
                break


        for data in json2['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
            try:
                text_box_ = data['node']

                date = datetime.datetime.fromtimestamp(int(data['node']['taken_at_timestamp'])).strftime('%Y-%m-%d %H:%M:%S')
                comment = text_box_['edge_media_to_caption']['edges'][0]['node']['text']
                comment = text_data(comment)
                idx_text = text_box_['id']
                uesr_id = text_box_['owner']['id']
                like = text_box_['edge_liked_by']['count']
                comments = [idx_text , uesr_id , date, comment , like , KEYWORD ]
#                 print(comments)

            except:
                comments = None
                pass


            try:
                insert_data(comments)
                commet_counts += 1
                pp = '완료 : {}'.foramt(commet_counts)
                print(pp , end = '\r')
            except:
                p = '중복이다 {}'.format(KEYWORD)
                print(p , end = '\r')
                
        print('성공이다 : {} , {}page'.format(KEYWORD , c) )

        c += 1
        try:
            max_id = json2['graphql']['hashtag']['edge_hashtag_to_media']['edges'][len(json2['graphql']['hashtag']['edge_hashtag_to_media']['edges']) - 1]['node']['id']
        except:
            print('끝났어')
            ERROR_COUNTS = MAX_ERROR
    
    return max_id


In [ ]:

ERROR_COUNTS = 0


FOOD_LIST = ['핫도그','곤약젤리','콩불','일본가정식','불닭','냉동삼겹살','방어','슈니발렌','후쿠오카함바그','대만카스테라','버블티','마라','펑리수','홍루이젠','명륜진사갈비','푸드트럭','딸기모찌']
COUNT_PER_70 = 300
DELAY_TIME = 0
MAX_ERROR = 50 #MAX_ERROR 만큼 에러나면 멈춘다


for KEYWORD in FOOD_LIST:

    max_id = get_first_json()
    stop_id = crawling_json(max_id)


In [14]:
ERROR_COUNTS = 0

KEYWORD = '훠궈'
COUNT_PER_70 = 300
DELAY_TIME = 0
MAX_ERROR = 5 


stop_id = '356233488100037078'

crawling_json(stop_id)


막혔어 기다려
그만 좀 들어와라 5분 쉰다
막혔어 기다려
그만 좀 들어와라 5분 쉰다
막혔어 기다려
그만 좀 들어와라 5분 쉰다
막혔어 기다려


KeyboardInterrupt: 

In [11]:
exception_delay(a)

False